In [1]:
import os
#Change the location
os.chdir("C:/Users/ahind/OneDrive/Desktop/Classes/Homework/Project")

# Import Module

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import nltk
import re
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#import os
#os.chdir("C:/Users/ahind/Downloads/")
from city_to_state import city_to_state_dict
two_word_states=["New Hampshire","New Jersey","New Mexico","New York","North Carolina","North Dakota","Rhode Island","South Carolina","South Dakota","West Virginia"]
import us
nltk.download('stopwords')
stopword = nltk.corpus.stopwords.words('english')


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ahind\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ahind\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Functions

In [3]:

def cleanUpTweet(row):
    import re
    # Remove mentions
    txt=row["text"]
    txt=txt.lower()
    row["hashtags"]=re.findall('(#[A-Za-z]+[A-Za-z0-9-_]+)', txt)
    row["hashtag_length"]=len(row["hashtags"])
    txt = re.sub(r'@[A-Za-z0-9_]+', '', txt)
    # Remove hashtags
    txt = re.sub(r'#[A-Z0-9]+', '', txt)
    # Remove retweets:
    txt = re.sub(r'RT : ', '', txt)
    # Remove urls
    txt = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', txt)
    #remove amp
    txt = re.sub(r'&amp;', '', txt)
    #rempve strange characters
    txt = re.sub(r'ðŸ™', '', txt)
    #remove new lines
    txt = re.sub(r'\n', ' ', txt)
    #removing punctuations
    txt = re.sub('[^\w\s]',' ',txt)
    #remove digits
    txt = re.sub('[d]+',' ',txt)
    #keeping only alphabets
    txt = re.sub("[^a-z]"," ", txt)
    #tockenisation
    text = re.split('\W+', txt)
   
    text = [word for word in text if word not in stopword]
    #stemming
    ps = nltk.PorterStemmer()
    row["tockens_without_stem"]=text
    row["tockens_without_stem"]=[word for word in row["tockens_without_stem"] if word not in ""]
    
    row["tockens"] = [ps.stem(word) for word in text]
    row["tockens"]=[word for word in row["tockens"] if word not in ""]
    row["Cleaned_text"]=" ".join([word for word in row["tockens"]])
    return row

def sentiment_ana(row):
    #for index, row in tweets_df['Cleaned_Text'].iteritems():
    text=row["Cleaned_text"]
    score = SentimentIntensityAnalyzer().polarity_scores(text)
    if score['neg'] > score['pos']:
        row["sentiment"]= "negative"
    elif score['pos'] > score['neg']:
        row["sentiment"]="positive"
    else:
        row["sentiment"]="neutral"
    

    row["neg"] = score['neg']
    row["neu"] = score['neu']
    row["pos"] = score['pos']
    row["compound"]= score['compound']
    return row

def loc_ana(loc):
    x=loc
    if re.match('({})'.format("|".join(two_word_states)), x.lower()):
        tokens = [re.match('({})'.format("|".join(two_word_states)), x.lower()).group(0)]
    elif re.match('({})'.format("|".join(city_to_state_dict.keys()).lower()), x.lower()):
        k = re.match('({})'.format("|".join(city_to_state_dict.keys()).lower()), x.lower()).group(0)
        tokens = [city_to_state_dict.get(k.title(), np.nan)]
    else:
        tokens = [j for j in re.split("\s|,", x) if j not in ['in', 'la', 'me', 'oh', 'or']]
    for i in tokens:
        if re.match('\w+', str(i)):
            if us.states.lookup(str(i)):
                 return us.states.lookup(str(i)).abbr
    return ""

# Main Code

In [5]:
import pandas as pd
chunksize = 1
temp = 1
cnt=1
error_index=[]
for chunk in pd.read_csv("data_2.csv", chunksize=chunksize):
    tweets_df=chunk
    #Extracting only relevant columns
    
    tweets_df = tweets_df[["id","created_at","text","user_location"]]
    tweets_df = tweets_df.rename(columns={"created_at": "date"})
    tweets_df['date'] = pd.to_datetime(tweets_df['date']).dt.date
    
    
        #Cleaning text
    #print(cnt)
    
        #Applying sentiment
    try:
        tweets_df=tweets_df.apply(cleanUpTweet,axis=1)
        
        tweets_df=tweets_df.apply(sentiment_ana,axis=1)
            #Extracting Location
        tweets_df["user_location"]=tweets_df["user_location"].fillna(" ")
        tweets_df["Location_clean"]=tweets_df["user_location"].apply(loc_ana)
    except:
        error_index.append(cnt)
        pass
        
    #chunk=chunk[[""]]
    if temp == 1:
        temp = 2
        tweets_df.to_csv('out_data_2_small.csv')
    else:
         tweets_df.to_csv('out_data_2_small.csv', mode='a', header=False)
    cnt=cnt+1